In [81]:
"""https://adventofcode.com/2022/day/5"""

from collections import deque
from dataclasses import dataclass
from typing import NamedTuple
from string import ascii_uppercase
from copy import deepcopy

test = """    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2"""

class Move(NamedTuple):
    quantity:int
    from_:int
    to:int
    
    @staticmethod
    def parse_move(line:str):
        """
        Parse text "move 1 from 2 to 1"
        into. Regex would be better noqa
        Move(quantity=1, from_=2, to=1)
        """
        line = line.strip() # safe
        qtt, mve = line.split('from')
        qtt = qtt.strip().split(" ")[-1]
        to = mve.strip().split('to')[-1].strip()
        from_ = mve.split(" ")[1]
        return Move(quantity=int(qtt),
                    from_=int(from_),
                    to=int(to))
@dataclass
class CargoCrane:
    inputs:str # our puzzle inputs
    
    def _parse_movements(self):
        """
        Convert puzzle input into list of Moves
        """
        t1 = self.inputs.split("\n\n")[1].split("\n")
        self.procedure = [Move.parse_move(line)
                          for line in t1]
    
    def _parse_stacks(self):
        """
        Convert puzzle input into intial stacks
        positions

        Cleans and loops over each row of data
        [Q] [J]                         [H]
        assigns letter to correct stack
        """
        t1 = self.inputs.split("\n\n")[0].split("\n")
        t2 = t1[:-1] # ignore last numbered row
        num_stacks = max(int(i) for i in t1[-1].strip().split()) # max num of stacks 
        # initiate stacks
        stacks = {i: deque()
                  for i in range(1,num_stacks+1)}
        window = deque() # for blanks
        for row in t2:
            ii = 1 # stack num
            L = len(row)
            for i in range(L):
                if row[i] == " ":
                    window.appendleft(row[i]) # increment blank window
                    if len(window) == 4: # 4 consecutive blanks -> empty crate in stack
                        stacks[ii].appendleft(row[i]) 
                        ii += 1
                        window = deque() # purge window
                elif row[i] in ascii_uppercase: # letter crate
                    stacks[ii].appendleft(row[i])
                    ii += 1
                    window = deque() # purge window
        # clean empty top crates
        stacks = {ii: [crate 
                       for crate in stack
                       if crate != " "]
                  for ii, stack in stacks.items()}
        self.initial_stacks = stacks
        
    def execute_procedure(self, crane9001=False):
        # parse
        self._parse_stacks()
        self._parse_movements()

        new_stacks = deepcopy(self.initial_stacks) # new stack list
        for move in self.procedure:
            temp_stack = list() # temp stack to hold crates in transit
            for _ in range(move.quantity):
                crate = new_stacks[move.from_].pop()
                temp_stack.append(crate)
            if crane9001: # part 2
                temp_stack = list(reversed(temp_stack))
            new_stacks[move.to].extend(temp_stack)
        self.current_stacks = new_stacks
        
    def top_crates(self)->str:
        """
        String of top crates on al stacks
        """
        top_crates = "".join(stack[-1]
                         for stack in self.current_stacks.values())
        return top_crates

# test
a = CargoCrane(inputs=test)
a.execute_procedure()

assert a.procedure == [Move(quantity=1, from_=2, to=1),
                       Move(quantity=3, from_=1, to=3),
                       Move(quantity=2, from_=2, to=1),
                       Move(quantity=1, from_=1, to=2)]

assert a.initial_stacks == {1: ['Z', 'N'], 2: ['M', 'C', 'D'], 3: ['P']}
assert a.current_stacks == {1: ['C'], 2: ['M'], 3: ['P', 'D', 'N', 'Z']}
assert a.top_crates() == 'CMZ'
# P2
a.execute_procedure(crane9001=True)
assert a.top_crates() == 'MCD'

with open('puzzle/day5.txt') as f:
    puzzle = f.read()
    cargo_crane = CargoCrane(inputs=puzzle)
    cargo_crane.execute_procedure()
    p1 = cargo_crane.top_crates()
    cargo_crane.execute_procedure(crane9001=True)
    p2 = cargo_crane.top_crates()
    print("p1->", p1)
    print("p2->", p2)    

p1-> VGBBJCRMN
p2-> LBBVJBRMH
